In [1]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("241205_03_reduce")
spark = SparkContext(conf=conf).getOrCreate()

24/12/05 14:41:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
#reduce 는 여러개의 값을 하나로 축약!!!

In [3]:
sample_rdd = spark.parallelize([1,2,3,4,5])
sample_rdd


ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [4]:
from operator import add

In [5]:
sample_rdd.reduce(add)

15

In [6]:
#partition 이 나뉘어 져 있을떄

#lambda x,y : (x*2)+y

#1: x=1, y=2 >> (1*2) +2

In [7]:
sample_rdd1 = spark.parallelize([1,2,3,4])
sample_rdd1.reduce(lambda x,y :(x*2)+y)

26

In [8]:
sample_rdd1.glom().collect()

[[1, 2, 3, 4]]

In [9]:
sample_rdd2 = spark.parallelize([1,2,3,4],2)
sample_rdd2.glom().collect()

[[1, 2], [3, 4]]

In [10]:
sample_rdd2.reduce(lambda x,y :(x*2)+y)

18

In [11]:
#리듀스 연산은 순서 의존적 
# 각 파티션 내에서 연산은 독립적으로 실행, 최종 결과는 파티션 간의 결과가 결합결과는 파티션 간의 결과가 결합

In [12]:
rdd4 = spark.parallelize([2,3,4],4)
rdd4.reduce(lambda x,y : x*y)

24

In [13]:
rdd4.fold(1,lambda x,y : x*y)

24

In [14]:
rdd5 = spark.parallelize([])
try :
    rdd5.reduce(lambda a,b : a+b)
except ValueError as e :
    print(f"Reduce Error : {e}")

Reduce Error : Can not reduce() empty RDD


In [15]:
rdd5.fold(0,lambda a,b : a+b)

0

In [16]:
rdd6 = spark.parallelize([1,"a",2])
rdd6.collect()

[1, 'a', 2]

In [17]:
try :
    rdd6.reduce(lambda x,y : x+y)
except TypeError as e:
    print(f"Reduce Error:{e}")

24/12/05 14:42:04 ERROR Executor: Exception in task 0.0 in stage 10.0 (TID 18)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/spark/python/pyspark/rdd.py", line 995, in func
    yield reduce(f, iterator, initial)
  File "/opt/spark/python/pyspark/util.py", line 73, in wrapper
    return f(*args, **kwargs)
  File "/tmp/ipykernel_4333/1636942717.py", line 2, in <lambda>
TypeError: unsupported operand type(s) for +: 'int' and 'str'

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:517)
	at org.apache.spark.api.python.PythonRunner$$

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 10.0 failed 1 times, most recent failure: Lost task 0.0 in stage 10.0 (TID 18) (ip-172-31-3-34.ap-northeast-3.compute.internal executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/spark/python/pyspark/rdd.py", line 995, in func
    yield reduce(f, iterator, initial)
  File "/opt/spark/python/pyspark/util.py", line 73, in wrapper
    return f(*args, **kwargs)
  File "/tmp/ipykernel_4333/

In [23]:
rdd6.fold("", lambda x,y : str(x)+str(y))

'1a2'

In [18]:
rdd7 = spark.parallelize([1,2,1,1,3,5,5,8])
rdd7.collect()

[1, 2, 1, 1, 3, 5, 5, 8]

In [19]:
result = rdd7.groupBy(lambda x: x%2).collect()

In [20]:
sorted([(x,sorted(y)) for (x,y) in result])

[(0, [2, 8]), (1, [1, 1, 1, 3, 5, 5])]

In [21]:
RDD.aggregate(zeroValue, func1_partion, func2_combine)

NameError: name 'RDD' is not defined

In [22]:
func1 = lambda x,y : (x[0]+y, x[1]+1)

In [23]:
func2 = lambda x,y : (x[0] +y[0], x[1]+y[1])

In [24]:
rdd8 = spark.parallelize([1,2,3,4],2)

In [25]:
rdd8.aggregate((0,0), func1, func2)

(10, 4)

In [26]:
rdd8.glom().collect()

[[1, 2], [3, 4]]

In [27]:
rdd9 = spark.parallelize(
    [
       ("짜장면",15),
       ("짬뽕",10),
       ("짜장면",5),
       ("짬뽕",20)
    ]
)

In [28]:
rdd9_g = rdd9.groupByKey()
rdd9_g

PythonRDD[30] at RDD at PythonRDD.scala:53

In [29]:
rdd9_g.mapValues(len).collect()

[('짜장면', 2), ('짬뽕', 2)]

In [30]:
rdd9_g.mapValues(list).collect()

[('짜장면', [15, 5]), ('짬뽕', [10, 20])]

In [31]:
rdd9_g.mapValues(lambda x : sum(x) / len(x)).collect()

[('짜장면', 10.0), ('짬뽕', 15.0)]

In [32]:
rdd9.reduceByKey(add).collect()

[('짜장면', 20), ('짬뽕', 30)]

In [33]:
rdd9.countByKey()

defaultdict(int, {'짜장면': 2, '짬뽕': 2})

In [34]:
rdd9.keys()

PythonRDD[40] at RDD at PythonRDD.scala:53

In [35]:
rdd9.keys().collect()

['짜장면', '짬뽕', '짜장면', '짬뽕']

In [36]:
join_rdd1 = spark.parallelize(
[    ("a",10),
    ("b",20),
    ("c",30),
] 
)

join_rdd2 = spark.parallelize(
[    ("a",20),
    ("b",20),
    ("c",10),
] 
)

join_rdd1.join(join_rdd2).collect()

[('b', (20, 20)), ('c', (30, 10)), ('a', (10, 20))]

In [37]:
join_rdd1.leftOuterJoin(join_rdd2).collect()

[('b', (20, 20)), ('c', (30, 10)), ('a', (10, 20))]

In [38]:
join_rdd1.rightOuterJoin(join_rdd2).collect()

[('b', (20, 20)), ('c', (30, 10)), ('a', (10, 20))]

In [39]:
spark.stop()